### Домашняя работа №1 по курсу Feature Engineering


In [95]:
import numpy as np
import pandas as pd
import random

from sklearn.linear_model import LinearRegression

#### Генерируем задачу.
Мы получили данные новых процессоров фирмы Outtel (c).
Известны колличество ядер в каждой модели, колличество операций ввода\вывода для каждого ядра и частота процессора. Спасибо тестировщикам! Их работа и уныла и важна! 
Необходимо построить модель линейной регрессии. 

In [96]:
n_samples = 555 # Число моделей процессоров в датасете

In [97]:
def get_model_name(temp):
    """ Функия назначения нового имени новой модели """
    vowel = 'eyuioa'
    consonant = 'qwwrtpsdfghjklzxcvbnm'
    model_name = 'Outtel-'
    for i in range (3):
        model_name  += random.choice(consonant)
        model_name  += random.choice(vowel)
    return model_name

Сгенерируем новый тестовый Датасет с операциями ввода\вывода для каждого ядра, частотой процессора и стоимостью, которая будет высчитываться по формуле COST = (сумма операций ввода\вывода всех ядер) / колличество ядер * частота процессора

In [90]:
core1 = np.random.choice(570, n_samples) + 430
core2 = np.random.choice(570, n_samples) + 430
core3 = np.random.choice(570, n_samples) + 430
core4 = np.random.choice(570, n_samples) + 430
core5 = np.random.choice(570, n_samples) + 430
core6 = np.random.choice(570, n_samples) + 430
frequency = np.random.choice(3000, n_samples) + 1500
price = np.around((core1+core2+core3+core4+core5+core6)/6 * frequency)
data = pd.DataFrame({'model_name': 'temp', 'core1': core1, 'core2': core2, 'core3': core3, 'core4': core4, 'core5': core5,
                     'core6': core6, 'frequency': frequency, 'price': price})
data['model_name']  = data['model_name'].apply(get_model_name)
data.head()

,model_name,core1,core2,core3,core4,core5,core6,frequency,price
0,Outtel-faxeze,923,472,814,803,972,598,2888,2205469.0
1,Outtel-lozere,998,831,818,649,682,479,3314,2461750.0
2,Outtel-wesela,495,768,680,624,540,630,3029,1886562.0
3,Outtel-buxoki,489,998,991,445,927,890,3247,2565130.0
4,Outtel-bytahe,691,876,934,529,951,596,4279,3264164.0


Попробуем высчитать линейную регресию в лоб.

In [91]:
from sklearn.metrics import mean_absolute_error

X = data[[ 'core1', 'core2', 'core3', 'core4', 'core5', 'core6']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['core1', 'core2', 'core3', 'core4', 'core5', 'core6']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [607.41853767 379.70680885 714.04020933 334.48710262 878.86451168
 594.72183185]
Bias: -384930.3165354626
Error: 547333.9708903851


Мы знаем, что что веса каждого ядра должны быть примерно одинаковы. Смещение и ошибка вообще за гранью. Значит - наша модель работает не так, как хотел заказчик.
Попробуем проанализировать имеющиеся данные. 
Цена зависит от производительности процессора, которая складывается из суммы производительностей каждого из шести ядер, помноженой на частоту процессора. 
Логичным было бы ввести дополнительный столбец расчёта произведения. 
performance = (core1+core2+core3+core4+core5+core6)/6  * frequency)

In [98]:
data['performance'] = (core1+core2+core3+core4+core5+core6)/6 * frequency

In [93]:
data.head()

,model_name,core1,core2,core3,core4,core5,core6,frequency,price,performance
0,Outtel-faxeze,923,472,814,803,972,598,2888,2205469.0,2.205469e+06
1,Outtel-lozere,998,831,818,649,682,479,3314,2461750.0,2.461750e+06
2,Outtel-wesela,495,768,680,624,540,630,3029,1886562.0,1.886562e+06
3,Outtel-buxoki,489,998,991,445,927,890,3247,2565130.0,2.565130e+06
4,Outtel-bytahe,691,876,934,529,951,596,4279,3264164.0,3.264164e+06


Попробуем построить линейную регрессию по новому параметру.

In [99]:
X = data[['performance']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['performance']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.99999997]
Bias: 0.06286634085699916
Error: 0.21130055003147571


#### Результат:
Оценив влияние наших данных на результат, и введя комплекснуый показатель оценики производительности, нам удалось подготовить данные для модели так, что отклонения и ошибка стали минимально допустимыми для нашей системы прогноза. 